In [158]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

In [2]:
data = pd.read_csv('../data/data_train.csv') #Load training set, please do not change this address.
data = data[~data.shot_made_flag.isna()] 
num_rows, num_columns = data.shape
column_names = data.columns
for col in column_names:
    print(col)

loc_x
loc_y
minutes_remaining
shot_distance
shot_made_flag
shot_id


In [3]:
X_train = data[['loc_x', 'loc_y']].values.astype(np.float32) #Load features from training set 
y_train = data['shot_made_flag'].values.astype(np.float32) #Load label from training set 
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

In [163]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        #######Please design your model here########
        self.hidden1 = nn.Linear(2, 4)
        self.ac1 = nn.ReLU()
        self.hidden2 = nn.Linear(4, 2)
        self.ac2 = nn.Tanh()
        self.output = nn.Linear(2, 1)
        self.ac3 = nn.Sigmoid()
        
    def forward(self, x):
        #######Please design your model here########
        x = self.ac1(self.hidden1(x))
        x = self.ac2(self.hidden2(x))
        x = self.ac3(self.output(x))
        return x

In [164]:
def train(model, optimizer, criterion, X_train_tensor, y_train_tensor):
    train_losses = []
    num_epochs = 400
    for epoch in range(num_epochs):
        # Training 
        model.train()
        
        #Forward calculation
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        
        #Backward training
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #Record the training loss 
        train_losses.append(loss.item())
        
        # Print the training loss
        model.eval()
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    return train_losses 

In [ ]:
model=MyModel()
criterion = nn.BCELoss()  # BCE Loss
optimizer = optim.AdamW(model.parameters(), lr=0.2)
train_losses = train(model, optimizer, criterion, X_train_tensor, y_train_tensor)

Epoch [100/400], Loss: 0.6713
Epoch [200/400], Loss: 0.6700
Epoch [300/400], Loss: 0.6681
Epoch [400/400], Loss: 0.6681


In [149]:
model_code = """  
import torch
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        #######Please design your model here########
        self.hidden1 = nn.Linear(2, 4)
        self.ac1 = nn.ReLU()
        self.hidden2 = nn.Linear(4, 2)
        self.ac2 = nn.Tanh()
        self.output = nn.Linear(2, 1)
        self.ac3 = nn.Sigmoid()
        
    def forward(self, x):
        #######Please design your model here########
        x = self.ac1(self.hidden1(x))
        x = self.ac2(self.hidden2(x))
        x = self.ac3(self.output(x))
        return x
"""
# Save the model structure
with open('submission_model.py', 'w') as f:
    f.write(model_code)
print("submission_model.py file is generated.")

submission_model.py file is generated.


In [150]:
# Save the model parameter
torch.save(model.state_dict(), 'submission_dic.pth')

In [151]:
import zipfile
import os

# Define the files to be packaged and the compressed file name. 
files_to_zip = ['submission_model.py', 'submission_dic.pth']
zip_filename = 'submission.zip'

# Create a zip file to submit.
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        # Add files to the zip file
        zipf.write(file, os.path.basename(file))

print(f'{zip_filename} is created successfully!')

submission.zip is created successfully!
